<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title"><b>Solving the Lego Planning Problem with Pyomo and Glpk</b></span> by <a xmlns:cc="http://creativecommons.org/ns#" href="http://mate.unipv.it/gualandi" property="cc:attributionName" rel="cc:attributionURL">Stefano Gualandi</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>. Based on a project at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/mathcoding/opt4ds" rel="dct:source">https://github.com/mathcoding/opt4ds</a>.

# 2.2 Solving the Lego Planning Problem

In this notebook, we show how to write the following **Linear Programming** model to solve the Lego Planning problem (see the slides on KIRO):

$$
\begin{align}
\max \quad & 8 c + 11 t  \\
 \quad & 2c + 2t \leq 24 \\
& c + 2 t \leq 18\\
& c \geq 0\\
& t \geq 0
\end{align}
$$


First, we need to install the [Gurobi](http://www.gurobi.org/). If you are running this notebook in a Colab, you don't need to install anything else on your computer.

The following line installs the free version of Gurobi in a Google Colab or on you computer.

In [ ]:
# Run if on Colab
# %pip install gurobipy

### 2.2.1 Define the model entities
To build the Linear Programming model with Gurobi, we need first to import the Gurobi python library:

In [1]:
from gurobipy import Model, GRB

At this point, we first declare a global object that refer to our model:

In [16]:
model = Model()

Then we star by declaring the two non/negative variables:

In [17]:
# declare decision variables (todo: read the docunmentation 
#   to understand the default values of parameters)
c = model.addVar(obj=8, name='x_c')
t = model.addVar(obj=11, name='x_t')

Here, we add variable `c` and `t` to the model. Notice that we are declaring the two variables $c \geq 0, t \geq 0$ (because of the default values of `addVar`).

We need to specify a direction for the objective function: min or max

In [18]:
model.ModelSense = GRB.MAXIMIZE

Next step is to introduce the two linear constraints:

In [19]:
# Declare constraints
smallPieces = model.addConstr(2*c + 2*t <= 24)
largePieces = model.addConstr(1*c + 2*t <= 18)

Notice that we are **declaring** the model, without programming any algorithm to actually solve this model. To find the optimal solution of this LP, we are going to use the Integer Linear Programming solver implemented in Gurobi.

Before we update our model (todo: check the documentation for [model.update]())

In [20]:
model.update()

### 2.2.2 Solve the model
We have use the type of objects of the Gurobi library to *declare* our Linear Programming model. Next, we have to use a solver to find the optimal values for the two decision variables.

**REMARK:** It is very important to learn to read the logs.

In [21]:
model.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x1140d349
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [8e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+01]
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9000000e+31   3.500000e+30   1.900000e+01      0s
       2    1.1400000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.140000000e+02


Every time we invoke a solver, it is very good practice to check the status of the solver, since it may have stop its execution for several different reasons:

In [22]:
# Basic info about the solution process
print(f"Status: {model.Status}, ObjVal: {model.ObjVal}")

Status: 2, ObjVal: 114.0


Whenever the status of the solver is 2 (=OK), we can query the solver to get the optimal value of the objective function and of the decision variables.

In [23]:
# Report solution value
print("Optimal solution value: z =", model.ObjVal)
print("Decision variables:")
print("\tProduction of chairs:", c.X)
print("\tProduction of tables:", t.X)

Optimal solution value: z = 114.0
Decision variables:
	Production of chairs: 6.0
	Production of tables: 6.0


As we found manually during the *Active Learning* session, the optimal production plan is to build 6 chairs and 6 tables, to get a profit of 114 kroner.

It is also possible to check the model passed to the solver with the following command:

In [10]:
# model.write('lego1.lp')

('lego1.lp', 4882522192)

### 2.2.3 Complete Script
We report below the complete script.

In [24]:
from gurobipy import Model, GRB

model = Model()

# declare decision variables (todo: read the docunmentation 
#   to understand the default values of parameters)
c = model.addVar(obj=8, name='x_c')
t = model.addVar(obj=11, name='x_t')

model.ModelSense = GRB.MAXIMIZE

# Declare constraints
smallPieces = model.addConstr(2*c + 2*t <= 24)
largePieces = model.addConstr(1*c + 2*t <= 18)

model.optimize()

# Basic info about the solution process
print(f"Status: {model.Status}, ObjVal: {model.ObjVal}")

# Report solution value
print("Optimal solution value: z =", model.ObjVal)
print("Decision variables:")
print("\tProduction of chairs:", c.X)
print("\tProduction of tables:", t.X)

# Basic info about the solution process
print(f"Status: {model.Status}, ObjVal: {model.ObjVal}")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x1140d349
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [8e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 2e+01]
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9000000e+31   3.500000e+30   1.900000e+01      0s
       2    1.1400000e+02   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.140000000e+02
Optimal solution value: z = 114.0
Decision variables:
	Production of chairs: 6.0
	Production of tables: 6.0
Status: 2, ObjVal: 114.0


## 2.2.4 Exercise: three variables
As an exercise, you have to modify the previous script to solve the second version of the Lego Planning problem, with a second type of danisk table:

$$
\begin{align}
\max \quad & 8 c + 11 t + 15 s \\
 \quad & 2c + 2t +2s \leq 24 \\
& c + 2 t +3s \leq 18\\
& c \geq 0\\
& t \geq 0\\
& s \geq 0
\end{align}
$$

Basically, you have to add a third variable to the model, to modify the objective function, and the two constraints. Later, you call the solver again.

In [ ]:
# TO BE COMPLETED ...